In [2]:
import pickle

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader
from torchinfo import summary

from tqdm.auto import tqdm

# own
import common.action as action
import common.world as world
import common.plot as plot
import common.preprocess as preprocess
import common.nets as nets
import common.train as train
import common.tools as tools

/Users/mariuskaestingschaefer/miniforge3/envs/pytorch/lib/python3.8/site-packages/seaborn/rcmod.py:82: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(mpl.__version__) >= "3.0":
/Users/mariuskaestingschaefer/miniforge3/envs/pytorch/lib/python3.8/site-packages/setuptools/_distutils/version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/Users/mariuskaestingschaefer/miniforge3/envs/pytorch/lib/python3.8/site-packages/torch/utils/tensorboard/__init__.py:6: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, '__version__') or LooseVersion(tensorboard.__version__) < LooseVersion('1.15'):


In [3]:
with open("datasets/oracle_data.pickle", "rb") as handle:
    oracle_data = pickle.load(handle)

with open("datasets/oracle_reversed_data.pickle", "rb") as handle:
    oracle_reversed_data = pickle.load(handle)

with open("datasets/oracle_random_data.pickle", "rb") as handle:
    oracle_random_data = pickle.load(handle)

In [4]:
length_trajectory = 10
batch_size = 128

# split and shuffle data
train_data, test_data = preprocess.split_data_for_trajectories(
    oracle_reversed_data, 0.8, length_trajectory
)
train_imgs, train_pos = preprocess.process_trajectory(train_data)
test_imgs, test_pos = preprocess.process_trajectory(test_data)

# stage data for the DataLoader
train_data = preprocess.ObtainDataset_notransform(train_imgs, train_pos)
test_data = preprocess.ObtainDataset_notransform(test_imgs, test_pos)

# DataLoader
dataset_loader_train_data = DataLoader(train_data, batch_size=batch_size, shuffle=True)
dataset_loader_test_data = DataLoader(test_data, batch_size=batch_size, shuffle=True)

In [7]:
inputs, label = next(iter(train_data))

print('len inputs', len(inputs))
print('len label', len(label))

print(inputs)
print(label)

# where is half of the coordinates? only use the LSTM to learn on coordinate?

len inputs 10
len label 10
[tensor([[[0.3740, 0.3793, 0.3800,  ..., 0.4017, 0.4313, 0.4507],
         [0.3698, 0.3771, 0.3814,  ..., 0.4091, 0.4209, 0.4332],
         [0.4057, 0.3962, 0.3856,  ..., 0.4180, 0.4315, 0.4176],
         ...,
         [0.4567, 0.4347, 0.4283,  ..., 0.4352, 0.4267, 0.4254],
         [0.4286, 0.4329, 0.4406,  ..., 0.4462, 0.4168, 0.4306],
         [0.4341, 0.4336, 0.4345,  ..., 0.4462, 0.4348, 0.4403]],

        [[0.2636, 0.2684, 0.2685,  ..., 0.3001, 0.3497, 0.3544],
         [0.2659, 0.2693, 0.2707,  ..., 0.3164, 0.3164, 0.3354],
         [0.3450, 0.3209, 0.2961,  ..., 0.3477, 0.3408, 0.2966],
         ...,
         [0.4236, 0.3653, 0.3028,  ..., 0.3591, 0.3354, 0.3157],
         [0.3837, 0.3313, 0.2828,  ..., 0.3337, 0.3026, 0.3350],
         [0.3877, 0.3317, 0.2790,  ..., 0.3257, 0.3412, 0.3179]],

        [[0.2237, 0.2280, 0.2290,  ..., 0.2606, 0.3066, 0.3058],
         [0.2279, 0.2304, 0.2315,  ..., 0.2761, 0.2716, 0.2901],
         [0.3046, 0.2823, 0.25